In [4]:
observation_angles_list = [
                    (11,0,12),
                    
                    (0,11,23),
                    (0,12,24),
                    
                    (11,13,15),
                    (13,15,17),
                    
                    (12,14,16),
                    (14,16,18),
                    
                    (11,23,25),
                    (12,24,26),
                    
                    (23,25,27),
                    (24,26,28),
                    
                    (25,27,29),
                    (26,28,30)
                    
                    ]

In [6]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

def read_keypoints_from_csv(csv_file):
    keypoints = []
    world_keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            frame_world_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
                
                world_x = float(row[133 + i*4])
                world_y = float(row[134 + i*4])
                world_z = float(row[135 + i*4])
                world_visibility = float(row[136 + i*4])
                frame_world_keypoints.append(landmark_pb2.NormalizedLandmark(x=world_x, y=world_y, z=world_z, visibility=world_visibility))
                
            keypoints.append(frame_keypoints)
            world_keypoints.append(frame_world_keypoints)
    return keypoints, world_keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

def angle_between_points(p1, p2, p3):
    # 벡터 정의
    vec1 = np.array(p1) - np.array(p2)
    vec2 = np.array(p3) - np.array(p2)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg


def compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1, world_keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2, world_keypoints2 = read_keypoints_from_csv(csv_file2)


    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    for angle in observation_angles_list:
        
        start_idx = angle[0]
        mid_idx = angle[1]
        finish_idx = angle[2]
        
        
        A1 = np.array([world_keypoints1[frame1_index][start_idx].x, world_keypoints1[frame1_index][start_idx].y, world_keypoints1[frame1_index][start_idx].z])
        B1 = np.array([world_keypoints1[frame1_index][mid_idx].x, world_keypoints1[frame1_index][mid_idx].y, world_keypoints1[frame1_index][mid_idx].z])
        C1 = np.array([world_keypoints1[frame1_index][finish_idx].x, world_keypoints1[frame1_index][finish_idx].y, world_keypoints1[frame1_index][finish_idx].z])

        A2 = np.array([world_keypoints2[frame2_index][start_idx].x, world_keypoints2[frame2_index][start_idx].y, world_keypoints2[frame2_index][start_idx].z])
        B2 = np.array([world_keypoints2[frame2_index][mid_idx].x, world_keypoints2[frame2_index][mid_idx].y, world_keypoints2[frame2_index][mid_idx].z])
        C2 = np.array([world_keypoints2[frame2_index][finish_idx].x, world_keypoints2[frame2_index][finish_idx].y, world_keypoints2[frame2_index][finish_idx].z])
        _, angle1 = angle_between_points(A1, B1, C1)
        _, angle2 = angle_between_points(A2, B2, C2)
        
        angle_distance = abs(angle1-angle2)
        if angle_distance > distance_threshold:
            highlight_indices.append(angle[1])
        
        print('angle_distance',angle_distance)

    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    for idx in highlight_indices:
        landmark = keypoints2[frame2_index][idx]
        cv2.circle(image2, (int(landmark.x * image2.shape[1]), int(landmark.y * image2.shape[0])), 20, (0, 0, 255), -1)

    combined_image = np.concatenate((image1, image2), axis=1)

    #cv2.putText(combined_image, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, f'threshold: {round(distance_threshold,3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'Angles', (1700, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, image2)


person1 = 'Z52'
person2 = 'Z106'
trial = '316'

# 사용 예시
video_file1 = f'C:/Users/jk/action_assess_2/data/video/{person1}/313-2-1-15-{person1}_D.avi'
video_file2 = f'C:/Users/jk/action_assess_2/data/video/{person2}/313-2-1-15-{person2}_D.avi'
csv_file1 = f'C:/Users/jk/action_assess_2/data/output/csv_key/{person1}/313-2-1-15-{person1}_D.csv'
csv_file2 = f'C:/Users/jk/action_assess_2/data/output/csv_key/{person2}/313-2-1-15-{person2}_D.csv'
output_image = 'C:/Users/jk/action_assess_2/post_data/angle/output_image.png'

for threshold in np.arange(10, 80, 10):
    output_image = f'C:/Users/jk/action_assess_2/post_data/angle/output_image_{round(threshold)}.png'
    compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=threshold)

angle_distance 0.8654288948688702
angle_distance 3.0754479397274963
angle_distance 3.4211917185990046
angle_distance 11.28762894356133
angle_distance 0.5224620457951517
angle_distance 4.656268278536572
angle_distance 8.946307895259196
angle_distance 19.898433809728218
angle_distance 1.9705123619104938
angle_distance 5.778180904694011
angle_distance 9.552396451957847
angle_distance 0.7149733518873802
angle_distance 1.6625855624741064
angle_distance 0.8654288948688702
angle_distance 3.0754479397274963
angle_distance 3.4211917185990046
angle_distance 11.28762894356133
angle_distance 0.5224620457951517
angle_distance 4.656268278536572
angle_distance 8.946307895259196
angle_distance 19.898433809728218
angle_distance 1.9705123619104938
angle_distance 5.778180904694011
angle_distance 9.552396451957847
angle_distance 0.7149733518873802
angle_distance 1.6625855624741064
angle_distance 0.8654288948688702
angle_distance 3.0754479397274963
angle_distance 3.4211917185990046
angle_distance 11.287628

In [26]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

def read_keypoints_from_csv(csv_file):
    keypoints = []
    world_keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            frame_world_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
                
                world_x = float(row[133 + i*4])
                world_y = float(row[134 + i*4])
                world_z = float(row[135 + i*4])
                world_visibility = float(row[136 + i*4])
                frame_world_keypoints.append(landmark_pb2.NormalizedLandmark(x=world_x, y=world_y, z=world_z, visibility=world_visibility))
                
            keypoints.append(frame_keypoints)
            world_keypoints.append(frame_world_keypoints)
    return keypoints, world_keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

def angle_between_points(p1, p2, p3):
    # 벡터 정의
    vec1 = np.array(p1) - np.array(p2)
    vec2 = np.array(p3) - np.array(p2)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg

def compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1, world_keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2, world_keypoints2 = read_keypoints_from_csv(csv_file2)

    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    angle_distances = []

    for i, angle in enumerate(observation_angles_list):
        start_idx = angle[0]
        mid_idx = angle[1]
        finish_idx = angle[2]
        
        A1 = np.array([world_keypoints1[frame1_index][start_idx].x, world_keypoints1[frame1_index][start_idx].y, world_keypoints1[frame1_index][start_idx].z])
        B1 = np.array([world_keypoints1[frame1_index][mid_idx].x, world_keypoints1[frame1_index][mid_idx].y, world_keypoints1[frame1_index][mid_idx].z])
        C1 = np.array([world_keypoints1[frame1_index][finish_idx].x, world_keypoints1[frame1_index][finish_idx].y, world_keypoints1[frame1_index][finish_idx].z])

        A2 = np.array([world_keypoints2[frame2_index][start_idx].x, world_keypoints2[frame2_index][start_idx].y, world_keypoints2[frame2_index][start_idx].z])
        B2 = np.array([world_keypoints2[frame2_index][mid_idx].x, world_keypoints2[frame2_index][mid_idx].y, world_keypoints2[frame2_index][mid_idx].z])
        C2 = np.array([world_keypoints2[frame2_index][finish_idx].x, world_keypoints2[frame2_index][finish_idx].y, world_keypoints2[frame2_index][finish_idx].z])
        
        _, angle1 = angle_between_points(A1, B1, C1)
        _, angle2 = angle_between_points(A2, B2, C2)
        
        angle_distance = abs(angle1 - angle2)
        angle_distances.append(angle_distance)
        
        if angle_distance > distance_threshold:
            highlight_indices.append(angle[1])
        
        print('angle_distance', angle_distance)

    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    for idx in highlight_indices:
        landmark = keypoints2[frame2_index][idx]
        cv2.circle(image2, (int(landmark.x * image2.shape[1]), int(landmark.y * image2.shape[0])), 20, (0, 0, 255), -1)

    combined_image = np.concatenate((image1, image2), axis=1)

    cv2.putText(combined_image, f'threshold: {round(distance_threshold, 3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'Angles', (1700, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, combined_image)

    return angle_distances

def normalize_angle_distances(angle_distances, max_angle=180):
    return [100 * (1 - (distance / max_angle) ) for distance in angle_distances]

def normalize_angle_distances_squared(angle_distances, max_angle=180):
    return [100 * (1 - (distance / max_angle) ** 2) for distance in angle_distances]

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def normalize_angle_distances_softmax(angle_distances):
    # Invert distances to make larger distances less significant
    inverted_distances = [1 / (distance + 1e-6) for distance in angle_distances]
    softmax_scores = softmax(inverted_distances)
    return [score * 100 for score in softmax_scores]

def round_values(values, decimals=2):
    return [round(value, decimals) for value in values]

person1 = 'Z52'
person2 = 'Z106'
output_csv_raw = 'C:/Users/jk/action_assess_2/post_data/angle/angle_distances_raw.csv'
output_csv_normalized = 'C:/Users/jk/action_assess_2/post_data/angle/angle_distances_normalized.csv'

# Write header to both CSVs
header = ['trial'] + [f'{angle[1]}_angle distance' for angle in observation_angles_list]
with open(output_csv_raw, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

with open(output_csv_normalized, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

for trial in range(313, 329):
    # 사용 예시
    video_file1 = f'C:/Users/jk/action_assess_2/data/video/{person1}/{trial}-2-1-15-{person1}_D.avi'
    video_file2 = f'C:/Users/jk/action_assess_2/data/video/{person2}/{trial}-2-1-15-{person2}_D.avi'
    csv_file1 = f'C:/Users/jk/action_assess_2/data/output/csv_key/{person1}/{trial}-2-1-15-{person1}_D.csv'
    csv_file2 = f'C:/Users/jk/action_assess_2/data/output/csv_key/{person2}/{trial}-2-1-15-{person2}_D.csv'
    output_image = f'C:/Users/jk/action_assess_2/post_data/angle/output_image_{trial}.png'
    
    angle_distances = compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=0.04)
    normalized_scores = normalize_angle_distances(angle_distances)
    
    # Round values to 2 decimal places
    angle_distances_rounded = round_values(angle_distances)
    normalized_scores_rounded = round_values(normalized_scores)
    
    # Append trial number and raw angle distances to raw CSV
    with open(output_csv_raw, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([trial] + angle_distances_rounded)
    
    # Append trial number and normalized scores to normalized CSV
    with open(output_csv_normalized, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([trial] + normalized_scores_rounded)
                

angle_distance 0.8654288948688702
angle_distance 3.0754479397274963
angle_distance 3.4211917185990046
angle_distance 11.28762894356133
angle_distance 0.5224620457951517
angle_distance 4.656268278536572
angle_distance 8.946307895259196
angle_distance 19.898433809728218
angle_distance 1.9705123619104938
angle_distance 5.778180904694011
angle_distance 9.552396451957847
angle_distance 0.7149733518873802
angle_distance 1.6625855624741064
angle_distance 4.868703495714968
angle_distance 7.731012503797018
angle_distance 6.86716437784537
angle_distance 5.084353843257958
angle_distance 11.989614072760986
angle_distance 25.238670507376042
angle_distance 10.5105842560327
angle_distance 23.049135766546925
angle_distance 19.764913517690033
angle_distance 1.371550241588821
angle_distance 2.310858283244059
angle_distance 0.029255403549427683
angle_distance 0.752636549882368
angle_distance 4.157102085540238
angle_distance 1.3247813285094594
angle_distance 4.35973360225708
angle_distance 18.973300720903

## appendix

In [6]:
import numpy as np

def angle_between_points(p1, p2, p3):
    # 벡터 정의
    vec1 = np.array(p1) - np.array(p2)
    vec2 = np.array(p3) - np.array(p2)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

def plot_points_and_lines(p1, p2, p3):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # 점들을 플롯
    ax.scatter(*p1, color='r', s=100, label='Point 1')
    ax.scatter(*p2, color='g', s=100, label='Point 2')
    ax.scatter(*p3, color='b', s=100, label='Point 3')
    
    # 점들을 선으로 연결
    ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='k')
    ax.plot([p2[0], p3[0]], [p2[1], p3[1]], [p2[2], p3[2]], color='k')
    
    # 축 레이블 설정
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # 범례 추가
    ax.legend()
    
    plt.show()

# 예시
p1 = [0, 1, 0]
p2 = [0, 2, 0]
p3 = [0, 2, 1/2]

plot_points_and_lines(p1, p2, p3)

angle_rad, angle_deg = angle_between_points(p1, p2, p3)
print(f"Angle in radians: {angle_rad}")
print(f"Angle in degrees: {angle_deg}")

Angle in radians: 1.5707963267948966
Angle in degrees: 90.0
